# Setup

Check: https://cutadapt.readthedocs.io/en/stable/guide.html?highlight=mixed#demultiplexing-paired-end-reads-in-mixed-orientation

In [1]:
# Activate environment
# conda activate qiime2-2021.4
conda activate qiime2-2022.2

# Go to root directory
if [ -z "${ROOT+x}" ]; then
  cd ../..
  export ROOT=true
fi
pwd

# export RESULTS_DIR="pair-adapters" POOL=CB1
export RESULTS_DIR="anchored" POOL=CB1
#export RESULTS_DIR="combinatorial" POOL=CB1
export OUTPUT_DIR="results/${RESULTS_DIR}/${POOL}"
mkdir -p $OUTPUT_DIR/demux

(qiime2-2022.2) (qiime2-2022.2) (qiime2-2022.2) (qiime2-2022.2) (qiime2-2022.2) (qiime2-2022.2) (qiime2-2022.2) 

: 1

# Demultiplex

In [2]:
# Pair-adapters
cutadapt \
--cores 8 \
--pair-adapters \
-g "file:metadata/${POOL}/^bc-fw.fasta" \
-G "file:metadata/${POOL}/^bc-rv.fasta" \
-o "${OUTPUT_DIR}/demux/round1-{name}-1.fastq.gz" \
-p "${OUTPUT_DIR}/demux/round1-{name}-2.fastq.gz" \
"datasets/${POOL}/forward.fastq.gz" "datasets/${POOL}/reverse.fastq.gz" \
--no-indels \
--errors 0

(qiime2-2022.2) 

: 1

In [ ]:
# Flipped
cutadapt -g file:barcodes.fasta \
-g "file:metadata/${POOL}/bc-fw.fasta" \
-G "file:metadata/${POOL}/bc-rv.fasta" \
-o "round2-{name}-2.fastq.gz" \
-p "round2-{name}-1.fastq.gz" \
round1-unknown-2.fastq.gz round1-unknown-1.fastq.gz

In [ ]:
# Combinatorial
cutadapt \
--cores 8 \
-g "file:metadata/${POOL}/bc-fw-idx.fasta" \
-G "file:metadata/${POOL}/bc-rv-idx.fasta" \
-o "${OUTPUT_DIR}/demux/{name1}-{name2}-1.fastq.gz" \
-p "${OUTPUT_DIR}/demux/{name1}-{name2}-2.fastq.gz" \
"datasets/${POOL}/forward.fastq.gz" "datasets/${POOL}/reverse.fastq.gz" \
--no-indels \
--errors 0 &> "${OUTPUT_DIR}/cutadapt.log"

In [ ]:
mmv < "metadata/${POOL}/mv-pattern.txt"

# Import

In [ ]:
cd ${OUTPUT_DIR} ||
qiime tools import \
    --type "SampleData[PairedEndSequencesWithQuality]" \
    --input-path manifest.csv \
    --input-format PairedEndFastqManifestPhred33 \
    --output-path "demux.qza"

In [ ]:
qiime demux summarize \
    --i-data "demux.qza" \
    --o-visualization "demux.qzv"

In [ ]:
qiime tools view "demux.qzv"

# Denoise and Filter

In [ ]:
qiime dada2 denoise-paired \
    --i-demultiplexed-seqs "demux.qza" \
    --p-trim-left-f 19 \
    --p-trim-left-r 20 \
    --p-trunc-len-f 200 \
    --p-trunc-len-r 200 \
    --p-n-threads 6 \
    --o-table "table.qza" \
    --o-representative-sequences "representative-sequences.qza" \
    --o-denoising-stats "denoising-stats.qza" \
    --verbose

In [ ]:
qiime metadata tabulate \
    --m-input-file "denoising-stats.qza" \
    --o-visualization "denoising-stats.qzv"

# Taxonomy

In [ ]:
# 16S
qiime feature-classifier classify-consensus-vsearch
    --i-query 'representative-sequences.qza' \
    --i-reference-reads 'silva-138-99-seqs.qza path' \
    --i-reference-taxonomy 'silva-138-99-tax.qza path' \
    --p-threads 6 \
    --o-classification '16S-taxonomy-vsearch.qza'

# ITS
qiime feature-classifier classify-consensus-vsearch \
    --i-query 'ITS-rep-seqs-dada2.qza path' \
    --i-reference-reads 'unite-8-99-seqs.qza path' \
    --i-reference-taxonomy 'unite-8-99-tax.qza path' \
    --p-threads 6 \
    --o-classification 'ITS-taxonomy-vsearch.qza'

In [ ]:
qiime taxa filter-table \
    --i-table "table.qza" \
    --i-taxonomy "taxonomy-v-search.qza" \
    --p-exclude "mitochondria,chloroplast" \
    --o-filtered-table "filtered-table.qza"

In [ ]:
qiime taxa filter-seqs \
    --i-sequences "representative-sequences.qzq" \
    --i-taxonomy "taxonomy-v-search.qza" \
    --p-exclude "mitochondria,chloroplast" \
    --o-filtered-sequences "filtered-sequences.qza"

# Visualizations

In [ ]:
qiime taxa barplot \
    --i-table "filtered-table.qza" \
    --i-taxonomy "taxonomy-vsearch.qza" \
    --m-metadata-file "mapping-file.txt" \
    --o-visualization "barplot.qzv"

qiime feature-table summarize \
    --i-table "filtered-table.qza" \
    --m-sample-metadata-file "mappingfile.tsv" \
    --o-visualization "feature-table.qzv"

qiime feature-table tabulate-seqs \
    --i-data "filtered-sequences.qza" \
    --o-visualization "sequence-table.qzv"

In [ ]:
## Philogeny
qiime phylogeny align-to-tree-mafft-fasttree \
    --i-sequences "rep-seqs-no-plastids.qza" \
    --output-dir "phylogeny"

## Diversity
qiime diversity core-metrics-phylogenetic \
    --i-table "table-no-plastids.qza" \
    --i-phylogeny "rooted_tree.qza" \
    --p-sampling-depth "median freq from feature-table summarize" \
    --m-metadata-file "mapping-file.txt" \
    --output-dir  "core-diversity-metrics"

## Alpha Rarefaction
qiime diversity alpha-rarefaction \
 --i-table "table-no-plastids.qza" \
 --i-phylogeny "rooted_tree.qza" \
 --p-max-depth "median freq from feature-table summary" \
 --m-metadata-file "mapping-file.txt" \
 --o-visualization alpha-rarefaction.qzv

## Group Significance
qiime diversity alpha-group-significance \
    --i-alpha-diversity "shannon_vector.qza" \
    --m-metadata-file "mappingfile.txt" \
    --o-visualization alpha-group-significance.qzv

qiime diversity beta-group-significance \
    --i-distance-matrix "bray_curtis_distance_matrix.qza" \
    --m-metadata-file "mapping file.txt" \
    --p-method "permanova" \
    --p-pairwise \
     --m-metadata-column "feature do compare" \
     --o-visualization "comparing-feature.qzv"